In [ ]:
pip install pmdarima

     |████████████████████████████████| 1.5MB 11.8MB/s 
     |████████████████████████████████| 2.1MB 32.0MB/s 
     |████████████████████████████████| 9.5MB 46.9MB/s 
  Found existing installation: Cython 0.29.21
    Uninstalling Cython-0.29.21:
      Successfully uninstalled Cython-0.29.21
  Found existing installation: statsmodels 0.10.2
    Uninstalling statsmodels-0.10.2:
      Successfully uninstalled statsmodels-0.10.2


In [24]:
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
import numpy as np
import seaborn as sns
import matplotlib as mpl
from datetime import datetime
import plotly.express as px
from statsmodels.tsa.stattools import adfuller
from statsmodels.tsa.seasonal import seasonal_decompose
from statsmodels.tsa.statespace.sarimax import SARIMAX
from statsmodels.tsa.stattools import kpss
from statsmodels.graphics.tsaplots import plot_acf
from statsmodels.graphics.tsaplots import plot_pacf
from pylab import rcParams
import pmdarima as pm
from sklearn.metrics import r2_score
mpl.rcParams['figure.figsize'] = (8,6)
mpl.rcParams['axes.grid'] = False
import warnings
warnings.filterwarnings("ignore")

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [ ]:
data = pd.read_csv("/content/gdrive/My Drive/Thesis/DMA_225.csv")

In [ ]:
data['Date-time'] = pd.to_datetime(data['Date-time'], dayfirst=True, errors='coerce')

In [ ]:
data.rename( columns={'Flow':'DMA_225','Date-time':'Date_time'}, inplace=True )

In [ ]:
data_DMA225 = data[['Date_time','DMA_225']]

In [ ]:
data_DMA225.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 35040 entries, 0 to 35039
Data columns (total 2 columns):
 #   Column     Non-Null Count  Dtype         
---  ------     --------------  -----         
 0   Date_time  35040 non-null  datetime64[ns]
 1   DMA_225    35032 non-null  float64       
dtypes: datetime64[ns](1), float64(1)
memory usage: 547.6 KB


In [ ]:
data_DMA225_indexed = data_DMA225.set_index('Date_time')

In [ ]:
dd = data_DMA225_indexed.resample('D').sum()

In [ ]:
dd

,DMA_225
Date_time,
2016-04-01,354.219001
2016-04-02,313.328000
2016-04-03,303.778001
2016-04-04,310.109000
2016-04-05,294.445000
...,...
2017-03-27,212.222000
2017-03-28,177.217999
2017-03-29,177.221000


In [ ]:
dd.shape

(365, 1)

In [ ]:
size_of_trainset = 0.7
split_value = round(len(dd)* size_of_trainset)

# Split
train = dd.iloc[:split_value]
test = dd.iloc[split_value:]

print(dd.shape)
print(train.shape,test.shape)

(365, 1)
(255, 1) (110, 1)


In [25]:
# Getting best arima model 
import itertools
p = d = q = range(0, 2)
pdq_cal = list(itertools.product(p, d, q))
# generation of different combinations of seasonal p, q and q triplets
pdq_seasonal = [(x[0], x[1], x[2], 7) for x in list(itertools.product(p, d, q))]
aic_bestvalue = np.inf
pdq_bestvalue = None
pdq_seasonal_best = None
tempmodel = None
for params in pdq_cal:   
    for seasonal_params in pdq_seasonal: 
        tempmodel = SARIMAX(train,order=params,seasonal_order = seasonal_params,enforce_invertibility=False,
                             enforce_stationarity=False)
        results = tempmodel.fit(disp=False)
        if results.aic < aic_bestvalue:
            aic_bestvalue = results.aic
            pdq_bestvalue = params
            pdq_seasonal_best = seasonal_params
print("Best ARIMA with seasonality (SARIMAX) {} x {} model - AIC:{}".format(pdq_bestvalue,pdq_seasonal_best,aic_bestvalue))


Best ARIMA with seasonality (SARIMAX) (1, 1, 1) x (1, 1, 1, 7) model - AIC:2802.513141419383


In [26]:
# using the best parameter in the sarimax model
sarimaxmodel = SARIMAX(train,order=(1,1,1),seasonal_order=(0,1,1,12),enforce_invertibility=False,
                             enforce_stationarity=False)
sarimaxmodel_fit = sarimaxmodel.fit()

In [27]:
sarimax_predictions = sarimaxmodel_fit.predict(start=len(train), end=len(train)+len(test)-1, dynamic=False)


In [28]:
#calculate rmse
from math import sqrt
from sklearn.metrics import mean_squared_error
# Root Mean Square Error (RMSE)
rmse = np.sqrt(mean_squared_error(test['DMA_225'], sarimax_predictions))
rmse = round(rmse, 3)


In [30]:
print("The rmse value of sarimax model is :" ,rmse)

The rmse value of sarimax model is : 148.411
